### Kaggle competition DSCC 483
### Mini Project - Fall 2022
### Tapan Pradyot
### Hailey Thanki



### This Notebook Contains the Primary Model used for generating the Final Kaggle Submission.

###### References:
###### https://www.nltk.org/api/nltk.tag.html
#### Please see relavant code sections for references


##### Importing libraries pandas and nltk and reading the csv file

In [ ]:
!pip install sentence_transformers

import pandas as pd
import matplotlib as plt
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer#for word embedding
import gensim
from gensim.models import Word2Vec
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import wordnet
from tqdm.notebook import tqdm
from tqdm.auto import trange
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer 
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


#### Enabling the GPU on the laptop
<b>reference:<b> https://www.tensorflow.org/guide/gpu

In [2]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)



1 Physical GPUs, 1 Logical GPUs


In [3]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


#### Reading the Training Dataset

In [4]:
df=pd.read_csv("training_data.csv")
df.head()

C:\Users\tapan\AppData\Local\Temp/ipykernel_25516/3588282187.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("training_data.csv")


,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us


##### converting the text column in the dataframe into a list with strings inside and then tokenizing the data

###### reference: https://stackoverflow.com/questions/41768196/python-convert-dataframe-into-a-list-with-string-items-inside-list
###### reference: https://www.analyticsvidhya.com/blog/2021/07/nltk-a-beginners-hands-on-guide-to-natural-language-processing/#:~:text=NLTK%20is%20a%20toolkit%20build,%2C%20parse%20tree%20visualization%2C%20etc%E2%80%A6

In [5]:
str_data=df['text'].astype(str).values.tolist()
token_data = [nltk.word_tokenize(i) for i in str_data]

##### storing the tokenized text as another column in the dataframe.

In [6]:
df['token_data']=token_data

##### pos-tagging and lemmatizing the Tokenized data and adding the lemmatized text as a column in the data frame
###### Reference: https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
###### Reference: https://towardsdatascience.com/building-a-text-normalizer-using-nltk-ft-pos-tagger-e713e611db8
###### Reference: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [7]:
df.head()

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country,token_data
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us,"[Remember, the, #, WuhanCoronaVirus, ?, The, p..."
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us,"[My, sources, @, WhiteHouse, say, 2, tactics, ..."
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us,"[I, 'll, venture, a, wild, guess, :, If, you, ..."
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us,"[#, Pakistan, (, #, GreenStimulus, =, #, Natur..."
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us,"[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past..."


In [8]:
from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV
lemmatized_data=[]
def mapping_pos(w):
    t = nltk.pos_tag([w])[0][1][0].upper()
    dict_pos_map = {
        # Look for N in the POS tag because all nouns begin with N
        'N': NOUN,
        # Look for V in the POS tag because all nouns begin with V
        'V':VERB,
        # Look for J in the POS tag because all nouns begin with J
        'J' : ADJ,
        # Look for R in the POS tag because all nouns begin with R
        'R':ADV  
    }
    return dict_pos_map.get(t, NOUN)


from nltk.stem import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
for i in tqdm(range(0, len(str_data)),desc="Progress"):
    x=[lmtzr.lemmatize(w, mapping_pos(w)) for w in nltk.word_tokenize(str_data[i])]
    lemmatized_data.append(x)
    
df['lemmatized_data']=lemmatized_data

Progress: 100%|██████████| 240000/240000 [33:05<00:00, 120.88it/s]


In [9]:
df.head()

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country,token_data,lemmatized_data
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us,"[Remember, the, #, WuhanCoronaVirus, ?, The, p...","[Remember, the, #, WuhanCoronaVirus, ?, The, p..."
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us,"[My, sources, @, WhiteHouse, say, 2, tactics, ...","[My, source, @, WhiteHouse, say, 2, tactic, wi..."
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us,"[I, 'll, venture, a, wild, guess, :, If, you, ...","[I, 'll, venture, a, wild, guess, :, If, you, ..."
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us,"[#, Pakistan, (, #, GreenStimulus, =, #, Natur...","[#, Pakistan, (, #, GreenStimulus, =, #, Natur..."
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us,"[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past...","[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past..."


In [10]:
df.to_csv('cleaned_lemmatized_data.csv')

##### removing the stopwords from the lemmatized text and adding filtered sentence in a column to the dataframe 
###### reference: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/


In [11]:
filtered_sentence = []
stop_words = set(stopwords.words('english'))

for i in tqdm(range(0,len(lemmatized_data)),desc="Progress"):
    sentence = lemmatized_data[i]
    filtered_sentence.append([w for w in sentence if not w in stop_words])

df['filtered_sentence']= filtered_sentence    

Progress: 100%|██████████| 240000/240000 [00:01<00:00, 158055.04it/s]


In [12]:
df.head()

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country,token_data,lemmatized_data,filtered_sentence
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us,"[Remember, the, #, WuhanCoronaVirus, ?, The, p...","[Remember, the, #, WuhanCoronaVirus, ?, The, p...","[Remember, #, WuhanCoronaVirus, ?, The, pandem..."
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us,"[My, sources, @, WhiteHouse, say, 2, tactics, ...","[My, source, @, WhiteHouse, say, 2, tactic, wi...","[My, source, @, WhiteHouse, say, 2, tactic, us..."
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us,"[I, 'll, venture, a, wild, guess, :, If, you, ...","[I, 'll, venture, a, wild, guess, :, If, you, ...","[I, 'll, venture, wild, guess, :, If, run, USA..."
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us,"[#, Pakistan, (, #, GreenStimulus, =, #, Natur...","[#, Pakistan, (, #, GreenStimulus, =, #, Natur...","[#, Pakistan, (, #, GreenStimulus, =, #, Natur..."
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us,"[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past...","[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past...","[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past..."


##### Removing numbers, words shorter than 2 chars, punctuations, links and emojis
##### Then joining the resulting cleaning text into a joined string (joined by space ' ')
###### reference: https://docs.python.org/3/howto/regex.html
###### reference: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b?permalink_comment_id=3203132
###### reference: https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate

In [13]:
clean_t = (df['filtered_sentence']).astype(str).to_list()
text_clean=[]

emoji = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]


for i in tqdm(range(0,len(filtered_sentence)),desc="Progress"):
    #removing words with characters less than 2
    clean=re.sub(r'\b\w{1,2}\b',"",clean_t[i])
    #removing punctuations
    clean=re.sub(r'[^\w\s]', '',clean)
    #removing numbers
    clean=re.sub(r'\d+', '', clean)
    #removing links
    clean=re.sub(r'http\S+', '', clean)   
    #removing emojis
    clean= emoji.sub(r'', clean)
    for punct in puncts:
            if punct in clean:
                    clean = clean.replace(punct,'')
        
    #joining the list of words back inot a joined string joined by space
    text_clean.append(clean)
    


Progress: 100%|██████████| 240000/240000 [00:09<00:00, 25699.59it/s]


##### adding the resulting clean text to the data frame as column text_clean

In [14]:
df['text_clean']=text_clean
df.head()

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country,token_data,lemmatized_data,filtered_sentence,text_clean
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us,"[Remember, the, #, WuhanCoronaVirus, ?, The, p...","[Remember, the, #, WuhanCoronaVirus, ?, The, p...","[Remember, #, WuhanCoronaVirus, ?, The, pandem...",Remember WuhanCoronaVirus The pandemic great...
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us,"[My, sources, @, WhiteHouse, say, 2, tactics, ...","[My, source, @, WhiteHouse, say, 2, tactic, wi...","[My, source, @, WhiteHouse, say, 2, tactic, us...",source WhiteHouse say tactic use get Americ...
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us,"[I, 'll, venture, a, wild, guess, :, If, you, ...","[I, 'll, venture, a, wild, guess, :, If, you, ...","[I, 'll, venture, wild, guess, :, If, run, USA...",venture wild guess run USA COVID crisis ...
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us,"[#, Pakistan, (, #, GreenStimulus, =, #, Natur...","[#, Pakistan, (, #, GreenStimulus, =, #, Natur...","[#, Pakistan, (, #, GreenStimulus, =, #, Natur...",Pakistan GreenStimulus Nature protection ...
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us,"[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past...","[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past...","[🇺🇸, Pandémie, de, #, coronavirus, :, 30, past...",Pandmie coronavirus pasteur amricains qui...


In [15]:
df.to_csv('cleaned_final_textCleaned_data.csv')

### Training the  Base Line Model using Logistic Regression and Naive Bayes

In [16]:
df=pd.read_csv("cleaned_final_textCleaned_data.csv",low_memory=False)
df=df.drop(columns=["Unnamed: 0"])
df.head()

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country,token_data,lemmatized_data,filtered_sentence,text_clean
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us,"['Remember', 'the', '#', 'WuhanCoronaVirus', '...","['Remember', 'the', '#', 'WuhanCoronaVirus', '...","['Remember', '#', 'WuhanCoronaVirus', '?', 'Th...",Remember WuhanCoronaVirus The pandemic great...
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us,"['My', 'sources', '@', 'WhiteHouse', 'say', '2...","['My', 'source', '@', 'WhiteHouse', 'say', '2'...","['My', 'source', '@', 'WhiteHouse', 'say', '2'...",source WhiteHouse say tactic use get Americ...
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us,"['I', ""'ll"", 'venture', 'a', 'wild', 'guess', ...","['I', ""'ll"", 'venture', 'a', 'wild', 'guess', ...","['I', ""'ll"", 'venture', 'wild', 'guess', ':', ...",venture wild guess run USA COVID crisis ...
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us,"['#', 'Pakistan', '(', '#', 'GreenStimulus', '...","['#', 'Pakistan', '(', '#', 'GreenStimulus', '...","['#', 'Pakistan', '(', '#', 'GreenStimulus', '...",Pakistan GreenStimulus Nature protection ...
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us,"['🇺🇸', 'Pandémie', 'de', '#', 'coronavirus', '...","['🇺🇸', 'Pandémie', 'de', '#', 'coronavirus', '...","['🇺🇸', 'Pandémie', 'de', '#', 'coronavirus', '...",Pandmie coronavirus pasteur amricains qui...


In [17]:
df['text_clean']

0         Remember  WuhanCoronaVirus  The pandemic great...
1          source  WhiteHouse say  tactic use get Americ...
2           venture wild guess   run USA  COVID crisis  ...
3          Pakistan   GreenStimulus   Nature protection ...
4          Pandmie   coronavirus   pasteur amricains qui...
                                ...                        
239995                     Likes  Retweets yentra    Mastеr
239996    Very interest Any thought   TheFive  Trump  KA...
239997     deal  COVID  forget  Christians   persecution...
239998    While hit   COVID death  President golf First ...
239999    This shall pas  Covid  May remain stand midst ...
Name: text_clean, Length: 240000, dtype: object

#### splitting the Training and Test data

In [19]:
df["text_clean"]=df["text_clean"].values.astype('U')

In [20]:
X_train, X_test, y_train, y_test = train_test_split(df["text_clean"],df["country"],test_size=0.2,shuffle=True)

In [21]:
t1_train = TfidfVectorizer(use_idf=True)
X_t1_train = t1_train.fit_transform(X_train)
X_t1_test = t1_train.transform(X_test)



### LogisticRegression

In [22]:
LR=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
LR.fit(X_t1_train, y_train)  #Predict y value for test dataset

y_predict = LR.predict(X_t1_test)
y_prob = LR.predict_proba(X_t1_test)[:,1]

print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))



              precision    recall  f1-score   support

   australia       0.48      0.44      0.46      7956
      canada       0.36      0.34      0.35      7955
     ireland       0.62      0.62      0.62      7999
 new_zealand       0.45      0.40      0.42      8023
          uk       0.33      0.34      0.33      8102
          us       0.41      0.52      0.46      7965

    accuracy                           0.44     48000
   macro avg       0.44      0.44      0.44     48000
weighted avg       0.44      0.44      0.44     48000

Confusion Matrix: [[3465  909  440  792  873 1477]
 [ 940 2696  689  989 1674  967]
 [ 371  617 4987  459  996  569]
 [ 729 1065  518 3185  962 1564]
 [ 813 1417  988  777 2723 1384]
 [ 861  690  407  899  959 4149]]


### Naive Bayes MultinomialNB

In [23]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB

In [24]:
gnb = MultinomialNB(alpha=0.7)
gnb.fit(X_t1_train, y_train)
y_predict = gnb.predict(X_t1_test)
y_prob = gnb.predict_proba(X_t1_test)[:,1]

print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))     
 

              precision    recall  f1-score   support

   australia       0.54      0.41      0.47      7956
      canada       0.41      0.29      0.34      7955
     ireland       0.57      0.65      0.60      7999
 new_zealand       0.43      0.40      0.41      8023
          uk       0.35      0.34      0.35      8102
          us       0.41      0.62      0.49      7965

    accuracy                           0.45     48000
   macro avg       0.45      0.45      0.44     48000
weighted avg       0.45      0.45      0.44     48000

Confusion Matrix: [[3262  645  621  904  743 1781]
 [ 744 2304  942 1141 1717 1107]
 [ 286  390 5175  571  883  694]
 [ 539  781  672 3223  914 1894]
 [ 633 1015 1268  865 2728 1593]
 [ 571  461  454  854  708 4917]]


## Training a LSTM Model (Final)

In [25]:
df=pd.read_csv("cleaned_final_textCleaned_data.csv",low_memory=False)
df=df.rename(columns={"Unnamed: 0": "index_no"})
df = df.set_index("index_no")
df["text_clean"]=df["text_clean"].values.astype('U')
df


,text,reply_to_screen_name,is_quote,is_retweet,hashtags,country,token_data,lemmatized_data,filtered_sentence,text_clean
index_no,,,,,,,,,,
0,Remember the #WuhanCoronaVirus? The pandemic w...,NaN,False,True,WuhanCoronaVirus KillerCuomo,us,"['Remember', 'the', '#', 'WuhanCoronaVirus', '...","['Remember', 'the', '#', 'WuhanCoronaVirus', '...","['Remember', '#', 'WuhanCoronaVirus', '?', 'Th...",Remember WuhanCoronaVirus The pandemic great...
1,My sources @WhiteHouse say 2 tactics will be u...,NaN,False,True,Trump,us,"['My', 'sources', '@', 'WhiteHouse', 'say', '2...","['My', 'source', '@', 'WhiteHouse', 'say', '2'...","['My', 'source', '@', 'WhiteHouse', 'say', '2'...",source WhiteHouse say tactic use get Americ...
2,I'll venture a wild guess: If you were running...,NaN,False,True,COVID19,us,"['I', ""'ll"", 'venture', 'a', 'wild', 'guess', ...","['I', ""'ll"", 'venture', 'a', 'wild', 'guess', ...","['I', ""'ll"", 'venture', 'wild', 'guess', ':', ...",venture wild guess run USA COVID crisis ...
3,#Pakistan (#GreenStimulus = #Nature protection...,NaN,False,True,Pakistan GreenStimulus Nature Green,us,"['#', 'Pakistan', '(', '#', 'GreenStimulus', '...","['#', 'Pakistan', '(', '#', 'GreenStimulus', '...","['#', 'Pakistan', '(', '#', 'GreenStimulus', '...",Pakistan GreenStimulus Nature protection ...
4,🇺🇸 Pandémie de #coronavirus: 30 pasteurs améri...,NaN,False,True,coronavirus COVID__19 COVIDー19,us,"['🇺🇸', 'Pandémie', 'de', '#', 'coronavirus', '...","['🇺🇸', 'Pandémie', 'de', '#', 'coronavirus', '...","['🇺🇸', 'Pandémie', 'de', '#', 'coronavirus', '...",Pandmie coronavirus pasteur amricains qui...
...,...,...,...,...,...,...,...,...,...,...
239995,"Aa Likes, Retweets yentra 🙏\n🔥🔥🔥\n#Mastеr",NaN,TRUE,TRUE,Mastеr,new_zealand,"['Aa', 'Likes', ',', 'Retweets', 'yentra', '🙏'...","['Aa', 'Likes', ',', 'Retweets', 'yentra', '🙏'...","['Aa', 'Likes', ',', 'Retweets', 'yentra', '🙏'...",Likes Retweets yentra Mastеr
239996,Very interesting\nAny thoughts?\n\n#TheFive #T...,NaN,FALSE,TRUE,TheFive Trump2020 KAG2020 mondaythoughts COVID...,new_zealand,"['Very', 'interesting', 'Any', 'thoughts', '?'...","['Very', 'interest', 'Any', 'thought', '?', '#...","['Very', 'interest', 'Any', 'thought', '?', '#...",Very interest Any thought TheFive Trump KA...
239997,As we deal with #COVID19 don't forget that #Ch...,NaN,TRUE,TRUE,COVID19 Christians persecution Nigeria,new_zealand,"['As', 'we', 'deal', 'with', '#', 'COVID19', '...","['As', 'we', 'deal', 'with', '#', 'COVID19', '...","['As', 'deal', '#', 'COVID19', ""n't"", 'forget'...",deal COVID forget Christians persecution...


In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["text_clean"])

In [27]:
def return_tokenized_text(train,column):
    
    total_words = len(tokenizer.word_index) + 1
    tokenized_text = []
    for line in tqdm(train[column]):
        tokenized_text.append(tokenizer.texts_to_sequences([line])[0])
    tokenized_test = []
    
    return tokenized_text,total_words

In [28]:
train_text,total_words = return_tokenized_text(df,"text_clean")

100%|██████████| 240000/240000 [00:03<00:00, 78441.94it/s]


In [29]:
total_words

394323

In [30]:
maxlen = max([len(x) for x in train_text])
maxlen=maxlen-1
padded_sequences_tr = pad_sequences(train_text,maxlen=maxlen,padding='post')
padded_sequences_te = pad_sequences(train_text,maxlen=maxlen,padding='post')

In [31]:
maxlen

47

In [32]:
le = LabelEncoder()
df["country"] = le.fit_transform(df['country'])

In [33]:
list(le.classes_)

['australia', 'canada', 'ireland', 'new_zealand', 'uk', 'us']

In [34]:
 le.transform(['australia', 'canada', 'ireland', 'new_zealand', 'uk', 'us'])

array([0, 1, 2, 3, 4, 5])

In [35]:
labels_train = to_categorical(df["country"])

#### Reference:
https://www.tensorflow.org/text/tutorials/text_classification_rnn

https://www.analyticsvidhya.com/blog/2021/06/lstm-for-text-classification/

In [36]:
def create_model(total_words, max_sequence_len):
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_len))
    model.add(Bidirectional(LSTM(150)))
    model.add(Dense(6,activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [37]:
maxlen


47

In [38]:
model_lstm = create_model(total_words, maxlen)

In [39]:
history = model_lstm.fit(padded_sequences_tr, labels_train, epochs=2, verbose=1)

Epoch 1/2
7500/7500 [==============================] - 168s 22ms/step - loss: 1.4600 - accuracy: 0.4126
Epoch 2/2
7500/7500 [==============================] - 162s 22ms/step - loss: 0.9042 - accuracy: 0.6753


### Reading and Cleaning the Test Data set(for Kaggle submission)

In [40]:
dft=pd.read_csv("test_data.csv")

str_data=dft['text'].astype(str).values.tolist()
token_data = [nltk.word_tokenize(i) for i in str_data]

dft['token_data']=token_data
dft.head()


,text,reply_to_screen_name,is_quote,is_retweet,hashtags,Id,token_data
0,"Ethical investing is not optional anymore, say...",NaN,False,False,covid19,0,"[Ethical, investing, is, not, optional, anymor..."
1,#COVID19 | Suite à la conférence de presse du ...,NaN,False,True,COVID19,1,"[#, COVID19, |, Suite, à, la, conférence, de, ..."
2,"Yesterday, I had a live discussion with @Steve...",NaN,False,True,COVID19,2,"[Yesterday, ,, I, had, a, live, discussion, wi..."
3,Nepal - #Coronavirus cases up 24% in a week. D...,NaN,False,True,Coronavirus,3,"[Nepal, -, #, Coronavirus, cases, up, 24, %, i..."
4,American economy jumped up a % big news story...,NaN,False,False,LysolAndCloroxSales,4,"[American, economy, jumped, up, a, %, big, new..."


In [41]:
from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV
lemmatized_data=[]
def mapping_pos(w):
    t = nltk.pos_tag([w])[0][1][0].upper()
    dict_pos_map = {
        # Look for N in the POS tag because all nouns begin with N
        'N': NOUN,
        # Look for V in the POS tag because all nouns begin with V
        'V':VERB,
        # Look for J in the POS tag because all nouns begin with J
        'J' : ADJ,
        # Look for R in the POS tag because all nouns begin with R
        'R':ADV  
    }
    return dict_pos_map.get(t, NOUN)


from nltk.stem import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
for i in tqdm(range(0, len(str_data)),desc="Progress"):
    x=[lmtzr.lemmatize(w, mapping_pos(w)) for w in nltk.word_tokenize(str_data[i])]
    lemmatized_data.append(x)
    
dft['lemmatized_data']=lemmatized_data

Progress: 100%|██████████| 60000/60000 [08:03<00:00, 124.10it/s]


In [42]:
dft.to_csv('Test_Data_lemmatized_data.csv')

In [43]:
filtered_sentence = []
stop_words = set(stopwords.words('english'))

for i in tqdm(range(0,len(lemmatized_data)),desc="Progress"):
    sentence = lemmatized_data[i]
    filtered_sentence.append([w for w in sentence if not w in stop_words])

dft['filtered_sentence']= filtered_sentence    

Progress: 100%|██████████| 60000/60000 [00:00<00:00, 322490.34it/s]


In [44]:
clean_t = (dft['filtered_sentence']).astype(str).to_list()
text_clean=[]

emoji = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]




for i in tqdm(range(0,len(filtered_sentence)),desc="Progress"):
    #removing words with characters less than 2
    clean=re.sub(r'\b\w{1,2}\b',"",clean_t[i])
    #removing punctuations
    clean=re.sub(r'[^\w\s]', '',clean)
    #removing numbers
    clean=re.sub(r'\d+', '', clean)
    #removing links
    clean=re.sub(r'http\S+', '', clean)   
    #removing emojis
    clean= emoji.sub(r'', clean)
    #joining the list of words back inot a joined string joined by space
    for punct in puncts:
            if punct in clean:
                    clean = clean.replace(punct,'')
    
    text_clean.append(clean)
    
dft['text_clean']=text_clean
dft.head()

dft.to_csv('Test_Data_final_Cleaned_data.csv')

Progress: 100%|██████████| 60000/60000 [00:02<00:00, 27281.66it/s]


### Prediciting the Country on the Test Data set using the the Model LSTM trained above

In [45]:
dft=pd.read_csv("Test_Data_final_Cleaned_data.csv",low_memory=False)
dft=dft.drop(columns=["Unnamed: 0"])
dft["text_clean"]=dft["text_clean"].values.astype('U')
dft

,text,reply_to_screen_name,is_quote,is_retweet,hashtags,Id,token_data,lemmatized_data,filtered_sentence,text_clean
0,"Ethical investing is not optional anymore, say...",NaN,False,False,covid19,0,"['Ethical', 'investing', 'is', 'not', 'optiona...","['Ethical', 'invest', 'be', 'not', 'optional',...","['Ethical', 'invest', 'optional', 'anymore', '...",Ethical invest optional anymore say SRI Juli...
1,#COVID19 | Suite à la conférence de presse du ...,NaN,False,True,COVID19,1,"['#', 'COVID19', '|', 'Suite', 'à', 'la', 'con...","['#', 'COVID19', '|', 'Suite', 'à', 'la', 'con...","['#', 'COVID19', '|', 'Suite', 'à', 'la', 'con...",COVID Suite confrence presse Premier min...
2,"Yesterday, I had a live discussion with @Steve...",NaN,False,True,COVID19,2,"['Yesterday', ',', 'I', 'had', 'a', 'live', 'd...","['Yesterday', ',', 'I', 'have', 'a', 'live', '...","['Yesterday', ',', 'I', 'live', 'discussion', ...",Yesterday live discussion SteveFDA COVID ...
3,Nepal - #Coronavirus cases up 24% in a week. D...,NaN,False,True,Coronavirus,3,"['Nepal', '-', '#', 'Coronavirus', 'cases', 'u...","['Nepal', '-', '#', 'Coronavirus', 'case', 'up...","['Nepal', '-', '#', 'Coronavirus', 'case', '24...",Nepal Coronavirus case week Deaths Tot...
4,American economy jumped up a % big news story...,NaN,False,False,LysolAndCloroxSales,4,"['American', 'economy', 'jumped', 'up', 'a', '...","['American', 'economy', 'jumped', 'up', 'a', '...","['American', 'economy', 'jumped', '%', 'big', ...",American economy jumped big news story whole...
...,...,...,...,...,...,...,...,...,...,...
59995,Who knew all we had to do was eat a tenner’s w...,NaN,False,True,Covid19,59995,"['Who', 'knew', 'all', 'we', 'had', 'to', 'do'...","['Who', 'knew', 'all', 'we', 'have', 'to', 'do...","['Who', 'knew', 'eat', 'tenner', '’', 'worth',...",Who knew eat tenner worth grub avoid contract...
59996,"Our death toll of more than 160,000 souls tell...",NaN,False,True,coronavirus,59996,"['Our', 'death', 'toll', 'of', 'more', 'than',...","['Our', 'death', 'toll', 'of', 'more', 'than',...","['Our', 'death', 'toll', '160,000', 'soul', 't...",Our death toll soul tell coronavirus never ...
59997,"Apparently, Donald Trump doesn't care about Am...",NaN,False,True,COVID19,59997,"['Apparently', ',', 'Donald', 'Trump', 'does',...","['Apparently', ',', 'Donald', 'Trump', 'do', ""...","['Apparently', ',', 'Donald', 'Trump', ""n't"", ...",Apparently Donald Trump care Americans live ...
59998,Is #HulaHooping the perfect #SocialDistancing ...,NaN,False,False,HulaHooping SocialDistancing London,59998,"['Is', '#', 'HulaHooping', 'the', 'perfect', '...","['Is', '#', 'HulaHooping', 'the', 'perfect', '...","['Is', '#', 'HulaHooping', 'perfect', '#', 'So...",HulaHooping perfect SocialDistancing activi...


In [46]:
test_text,total_words = return_tokenized_text(dft,"text_clean")

100%|██████████| 60000/60000 [00:00<00:00, 73174.48it/s]


In [47]:
padded_sequences_te = pad_sequences(test_text,maxlen=maxlen,padding='post')

In [48]:
pred = model_lstm.predict(padded_sequences_te)

In [49]:
pred = np.argmax(pred, axis=-1)

In [50]:
final = pd.DataFrame()
final['id'] = dft['Id']
final_prediction = le.inverse_transform(pred)
final['category'] = final_prediction
final.to_csv('Test_Data_predictions_data.csv',index=False)